In [ ]:
# ===================================================================
# 0. 필요 라이브러리 설치
# ===================================================================
!pip install -q roboflow
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 109.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 121.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3

In [ ]:
import os
import torch
from ultralytics import YOLO
import roboflow
from getpass import getpass
from roboflow import Roboflow


# ===================================================================
# 1. COCO 데이터셋 준비 (사람, 고양이, 개 클래스만 포함)
# ===================================================================

# Roboflow API 키 입력받기
try:
    api_key = getpass("Roboflow API 키를 입력하세요: ")
except Exception as e:
    print("getpass를 실행할 수 없습니다. API 키를 직접 코드에 입력해주세요.")
    api_key = "" # 여기에 직접 API 키를 붙여넣으세요.

# Roboflow에 연결하여 데이터셋 다운로드
try:
    rf = roboflow.Roboflow(api_key=api_key) #
    project = rf.workspace("smartcarparkingsystem").project("person-cat-dog")
    dataset = project.version(1).download("yolov8")
    DATASET_PATH = dataset.location
except Exception as e:
    print(f"데이터셋 다운로드 중 오류 발생: {e}")
    print("수동으로 https://universe.roboflow.com/datasets/coco-person-cat-dog 에서 다운로드 후")
    print("DATASET_PATH 변수에 해당 경로를 직접 지정해주세요.")
    DATASET_PATH = "./COCO-Person-Cat-Dog-1" # 다운로드 실패 시 이 경로를 수정하세요.

# 데이터셋 설정 파일 경로 확인
data_yaml_path = os.path.join(DATASET_PATH, 'data.yaml')
print(f"✅ 데이터셋 준비 완료! 경로: {DATASET_PATH}")
print("-" * 30)


Roboflow API 키를 입력하세요: ··········
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Person-Cat-Dog-1 in yolov8:: 100%|██████████| 18158/18158 [00:03<00:00, 5296.81it/s]


✅ 데이터셋 준비 완료! 경로: /content/Person-Cat-Dog-1
------------------------------


In [ ]:

# ===================================================================
# 2. YOLOv8 모델 학습 (yolov8n.pt 사용)
# ===================================================================
print("🚀 YOLOv8n 모델 학습을 시작합니다...")

# 사전 학습된 yolov8n 모델 로드
model_v8 = YOLO('yolov8n.pt')

# 모델 학습
# epochs: 전체 데이터셋을 몇 번 반복 학습할지 결정 (시연을 위해 15로 설정)
# imgsz: 학습에 사용할 이미지 크기
# device: 'cpu' 또는 '0' (첫 번째 GPU)
# project/name: 학습 결과가 저장될 폴더 이름
results_v8 = model_v8.train(
    data=data_yaml_path,
    epochs=20,
    imgsz=640,
    augment=True,
    lr0=0.001,
    project='runs/train',
    name='yolov8n_person_cat_dog',
    exist_ok=True # 이미 폴더가 있어도 덮어쓰기 허용
)
print("✅ YOLOv8n 모델 학습 완료!")
print(f"결과는 '{results_v8.save_dir}' 폴더에 저장되었습니다.")
print("-" * 30)



🚀 YOLOv8n 모델 학습을 시작합니다...


100%|██████████| 6.25M/6.25M [00:00<00:00, 111MB/s]


Ultralytics 8.3.169 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Person-Cat-Dog-1/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8n_person_cat_dog, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True

100%|██████████| 755k/755k [00:00<00:00, 19.5MB/s]

Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      20/20      3.04G     0.8161     0.7497      1.189         21        640: 100%|██████████| 397/397 [01:43<00:00,  3.84it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 57/57 [00:14<00:00,  3.92it/s]


                   all       1816       4580      0.864      0.778      0.871      0.668

20 epochs completed in 0.688 hours.
Optimizer stripped from runs/train/yolov8n_person_cat_dog/weights/last.pt, 6.2MB
Optimizer stripped from runs/train/yolov8n_person_cat_dog/weights/best.pt, 6.2MB

Validating runs/train/yolov8n_person_cat_dog/weights/best.pt...
Ultralytics 8.3.169 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 57/57 [00:22<00:00,  2.56it/s]


                   all       1816       4580      0.851      0.779      0.871      0.685
                   cat        518        947      0.848      0.832      0.916       0.75
                   dog        502        939      0.864      0.869      0.933        0.8
                person       1075       2694       0.84      0.636      0.765      0.505
Speed: 0.2ms preprocess, 6.6ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs/train/yolov8n_person_cat_dog
✅ YOLOv8n 모델 학습 완료!
결과는 'runs/train/yolov8n_person_cat_dog' 폴더에 저장되었습니다.
------------------------------


In [ ]:

# ===================================================================
# 3. 모델 성능 비교
# ===================================================================
print("🔍 모델 성능 비교를 시작합니다...")

# --- 3.1. 방금 학습한 YOLOv8 모델 성능 평가 ---
# 가장 성능이 좋았던 가중치(best.pt)의 경로
best_v8_model_path = os.path.join(results_v8.save_dir, 'weights/best.pt')
best_v8_model = YOLO(best_v8_model_path)

# 검증 데이터셋으로 성능 평가
print("\n--- [YOLOv8n-custom] 성능 평가 결과 ---")
metrics_v8 = best_v8_model.val(
    data=data_yaml_path,
    split='val',
    project='runs/eval',
    name='yolov8n_eval'
)
# mAP50-95 점수는 metrics_v8.box.map 입니다.
# 클래스별 상세 정보는 자동으로 출력됩니다.

print("-" * 30)




# --- 3.2. 기존 YOLOv5s 모델 성능 평가 (동일한 데이터셋으로) ---
print("\n--- [YOLOv5s-pretrained] 성능 평가 결과 ---")
# YOLOv5s 모델 로드 (ultralytics 라이브러리는 v5도 지원)
model_v5 = YOLO('yolov5s.pt')

# 동일한 검증 데이터셋으로 성능 평가
# 이 과정이 두 모델을 공정하게 비교하는 핵심입니다.
metrics_v5 = model_v5.val(
    data=data_yaml_path,
    split='val',
    project='runs/eval',
    name='yolov5s_eval'
)
# mAP50-95 점수는 metrics_v5.box.map 입니다.

print("✅ 모든 성능 평가 완료!")
print("-" * 30)


# ===================================================================
# 4. 최종 결과 요약
# ===================================================================
print("\n\n📊 최종 성능 비교 요약")
print("==========================================")
print(f"모델\t\t\t| mAP50-95   | mAP50")
print("------------------------------------------")
print(f"YOLOv5s (Pretrained)\t| {metrics_v5.box.map:.4f}     | {metrics_v5.box.map50:.4f}")
print(f"YOLOv8n (Custom Trained)\t| {metrics_v8.box.map:.4f}     | {metrics_v8.box.map50:.4f}")
print("==========================================")
print("\n* mAP50-95: IoU 50%~95% 기준 평균 mAP (핵심 성능 지표)")
print("* mAP50: IoU 50% 기준 mAP (탐지를 얼마나 잘하는지 보는 너그러운 지표)")

🔍 모델 성능 비교를 시작합니다...

--- [YOLOv8n-custom] 성능 평가 결과 ---
Ultralytics 8.3.169 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1221.1±754.8 MB/s, size: 48.3 KB)


val: Scanning /content/Person-Cat-Dog-1/valid/labels.cache... 1816 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1816/1816 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 114/114 [00:18<00:00,  6.28it/s]


                   all       1816       4580      0.864      0.778       0.87      0.667
                   cat        518        947      0.863       0.83      0.908      0.725
                   dog        502        939      0.881      0.864       0.93       0.78
                person       1075       2694      0.849      0.639      0.773      0.496
Speed: 0.7ms preprocess, 3.8ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs/eval/yolov8n_eval
------------------------------

--- [YOLOv5s-pretrained] 성능 평가 결과 ---
PRO TIP 💡 Replace 'model=yolov5s.pt' with new 'model=yolov5su.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.



100%|██████████| 17.7M/17.7M [00:00<00:00, 116MB/s] 


Ultralytics 8.3.169 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv5s summary (fused): 84 layers, 9,142,496 parameters, 0 gradients, 24.0 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1931.0±524.3 MB/s, size: 65.3 KB)


val: Scanning /content/Person-Cat-Dog-1/valid/labels.cache... 1816 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1816/1816 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 114/114 [00:22<00:00,  5.13it/s]


                   all       1816       4580     0.0226      0.309      0.016    0.00944
                person        518        947     0.0118      0.697     0.0159    0.00897
               bicycle        502        939     0.0438      0.166     0.0256     0.0164
                   car       1075       2694     0.0122      0.065    0.00651    0.00299
Speed: 0.4ms preprocess, 7.9ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs/eval/yolov5s_eval
✅ 모든 성능 평가 완료!
------------------------------


📊 최종 성능 비교 요약
모델			| mAP50-95   | mAP50
------------------------------------------
YOLOv5s (Pretrained)	| 0.0094     | 0.0160
YOLOv8n (Custom Trained)	| 0.6671     | 0.8704

* mAP50-95: IoU 50%~95% 기준 평균 mAP (핵심 성능 지표)
* mAP50: IoU 50% 기준 mAP (탐지를 얼마나 잘하는지 보는 너그러운 지표)
